In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr 
import yfinance as yf
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error,r2_score
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
df = yf.download(tickers = "EURUSD=X", period = "10y", interval = "1d", group_by = 'ticker', auto_adjust = True,
                   prepost = True, threads = True, proxy = None)
df =df.drop(['Volume'],1)
df =df.drop(['Open'],1)
df =df.drop(['High'],1)
df =df.drop(['Low'],1)
df['LogClose'] = np.log(df['Close'])
df['DiffLogClose'] = df['LogClose'].diff()
df = df[1:]
df = df[:-1]
last_train = df.iloc[-1]['LogClose']
df

[*********************100%***********************]  1 of 1 completed


C:\Users\Kazem Kamrani\AppData\Local\Temp\ipykernel_12520\304943898.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df =df.drop(['Volume'],1)
C:\Users\Kazem Kamrani\AppData\Local\Temp\ipykernel_12520\304943898.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df =df.drop(['Open'],1)
C:\Users\Kazem Kamrani\AppData\Local\Temp\ipykernel_12520\304943898.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df =df.drop(['High'],1)
C:\Users\Kazem Kamrani\AppData\Local\Temp\ipykernel_12520\304943898.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df =df.drop(['Low'],1)


,Close,LogClose,DiffLogClose
Date,,,
2012-10-11,1.285843,0.251414,0.000000
2012-10-12,1.293326,0.257218,0.005803
2012-10-15,1.294331,0.257994,0.000776
2012-10-16,1.296849,0.259937,0.001943
2012-10-17,1.310788,0.270628,0.010691
...,...,...,...
2022-10-03,0.979269,-0.020949,-0.003758
2022-10-04,0.983207,-0.016936,0.004013
2022-10-05,0.998432,-0.001569,0.015367


In [3]:
for i in range(1, 7):
    df[f"DiffLogClose{i}"] = df["DiffLogClose"].shift(-i)
    
df = df[:-6]

df =df.drop(['Close'],1)
df =df.drop(['LogClose'],1)  


C:\Users\Kazem Kamrani\AppData\Local\Temp\ipykernel_12520\1581316184.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df =df.drop(['Close'],1)
C:\Users\Kazem Kamrani\AppData\Local\Temp\ipykernel_12520\1581316184.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df =df.drop(['LogClose'],1)


In [4]:
forecast_out = int(1)
df['PredictionDiffLogClose'] = df['DiffLogClose6'].shift(-forecast_out)
print(df)

            DiffLogClose  DiffLogClose1  DiffLogClose2  DiffLogClose3  \
Date                                                                    
2012-10-11      0.000000       0.005803       0.000776       0.001943   
2012-10-12      0.005803       0.000776       0.001943       0.010691   
2012-10-15      0.000776       0.001943       0.010691       0.000656   
2012-10-16      0.001943       0.010691       0.000656      -0.003614   
2012-10-17      0.010691       0.000656      -0.003614      -0.003574   
...                  ...            ...            ...            ...   
2022-09-23      0.001102      -0.015527      -0.006856      -0.002864   
2022-09-26     -0.015527      -0.006856      -0.002864       0.011602   
2022-09-27     -0.006856      -0.002864       0.011602       0.012426   
2022-09-28     -0.002864       0.011602       0.012426      -0.003758   
2022-09-29      0.011602       0.012426      -0.003758       0.004013   

            DiffLogClose4  DiffLogClose5  DiffLogC

In [5]:
X = np.array(df.drop(['PredictionDiffLogClose'],1))
#X = preprocessing.scale(X)
X_forecast = X[-forecast_out:]
X = X[:-forecast_out]

C:\Users\Kazem Kamrani\AppData\Local\Temp\ipykernel_12520\523623850.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = np.array(df.drop(['PredictionDiffLogClose'],1))


In [6]:
Y = np.array(df['PredictionDiffLogClose'])
Y = Y[:-forecast_out]

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [8]:
# Training
LR = LinearRegression()
LR.fit(X_train, Y_train)
testpredictions = LR.predict(X_test)
#Testing
confidence = LR.score(X_test,Y_test)
print("confidence:", confidence)
print('R2 Score is: ', metrics.explained_variance_score(Y_test, testpredictions))
print('Mean Squared Error is: ', metrics.mean_squared_error(Y_test, testpredictions))
print('Root Mean Squared Error is: ', np.sqrt(metrics.mean_squared_error(Y_test, testpredictions)))
print('Mean Absolute Error is: ', metrics.mean_absolute_error(Y_test, testpredictions))
print('Mean Absolute Percentage Error is: ', metrics.mean_absolute_percentage_error(Y_test, testpredictions))

confidence: -0.006331384655240635
R2 Score is:  0.002247046590566404
Mean Squared Error is:  2.5274056727851307e-05
Root Mean Squared Error is:  0.005027330974568047
Mean Absolute Error is:  0.003686038311302076
Mean Absolute Percentage Error is:  4673938574.479123


In [9]:
forecast_prediction = LR.predict(X_forecast)
print(forecast_prediction)

[-0.0011325]


In [10]:
forecast_Price = 10 ** (last_train + forecast_prediction)
forecast_Price

array([0.94961236])

In [16]:
qq = exp( last_train)
qq

NameError: name 'exp' is not defined

In [13]:
b = np.log(0.978905)
b

-0.02132067895376495

In [14]:
last_train

-0.02132114344515707